# Projeto 7: Classificação multiclasse iris com validação cruzada

## Etapa 1: Importação das bibliotecas

In [ ]:
!pip install skorch

     |████████████████████████████████| 133kB 4.5MB/s 


In [ ]:
import pandas as pd
import numpy as np
import torch.nn as nn        
from skorch import NeuralNetClassifier
import torch
from sklearn.model_selection import cross_val_score
torch.__version__

'1.6.0+cu101'

## Etapa 2: Base de dados

In [ ]:
np.random.seed(123)
torch.manual_seed(123)

In [ ]:
base = pd.read_csv('iris.csv')
previsores = base.iloc[:, 0:4].values
classe = base.iloc[:, 4].values

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
classe = encoder.fit_transform(classe)

In [ ]:
np.unique(classe)

array([0, 1, 2])

In [ ]:
previsores = previsores.astype('float32')
classe = classe.astype('int64')

## Etapa 3: Construção do modelo

In [ ]:
class classificador_torch(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense0 = nn.Linear(4, 16)
        self.activation0 = nn.ReLU()
        self.dense1 = nn.Linear(16, 16)
        self.activation1 = nn.ReLU()
        self.dense2 = nn.Linear(16, 3)
        
    def forward(self, X):
        X = self.dense0(X)
        X = self.activation0(X)
        X = self.dense1(X)
        X = self.activation1(X)
        X = self.dense2(X)
        return X

In [ ]:
classificador_sklearn = NeuralNetClassifier(module = classificador_torch, 
                                            criterion = torch.nn.CrossEntropyLoss, 
                                            optimizer = torch.optim.Adam,
                                            max_epochs = 1000,
                                            batch_size = 10,
                                            train_split = False)

## Etapa 4: Validação cruzada

In [ ]:
resultados = cross_val_score(classificador_sklearn, previsores, classe, cv = 5, 
                             scoring = 'accuracy')

A saída de streaming foi truncada nas últimas 5000 linhas.
     10        0.4109  0.0187
     11        0.5542  0.0185
     12        0.4098  0.0190
     13        0.5332  0.0178
     14        0.4993  0.0185
     15        0.3979  0.0181
     16        0.3369  0.0186
     17        0.3687  0.0185
     18        0.3944  0.0199
     19        0.2903  0.0188
     20        0.3097  0.0174
     21        0.2867  0.0202
     22        0.2556  0.0174
     23        0.2571  0.0232
     24        0.2474  0.0162
     25        0.2309  0.0174
     26        0.2160  0.0173
     27        0.2134  0.0160
     28        0.2090  0.0183
     29        0.2031  0.0170
     30        0.1998  0.0194
     31        0.1923  0.0217
     32        0.1882  0.0201
     33        0.1818  0.0218
     34        0.1826  0.0194
     35        0.1794  0.0187
     36        0.1772  0.0165
     37        0.1729  0.0205
     38        0.1697  0.0169
     39        0.1644  0.0187
     40        0.1669  0.0186
     41    

In [ ]:
media = resultados.mean()
desvio = resultados.std()

In [ ]:
media, desvio

(0.9800000000000001, 0.02666666666666666)

In [ ]:
resultados

array([1.        , 1.        , 0.96666667, 0.93333333, 1.        ])

# Tunning 

In [ ]:
# Tunning 

import pandas as pd
import torch.nn as nn
from skorch import NeuralNetClassifier
import torch
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV
torch.__version__

class classificador_torch(nn.Module):
    def __init__(self, activation, neurons, initializer, dropout):
        super().__init__()
        self.dense0 = nn.Linear(4, neurons)
        initializer(self.dense0.weight)
        self.activation0 = activation
        self.dense1 = nn.Linear(neurons, neurons)
        initializer(self.dense1.weight)
        self.activation1 = activation
        self.dense2 = nn.Linear(neurons, 3)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, X):
        X = self.dense0(X)
        X = self.activation0(X)
        X = self.dropout(X)
        X = self.dense1(X)
        X = self.activation1(X)
        X = self.dropout(X)
        X = self.dense2(X)
        return X

In [ ]:
torch.manual_seed(123)
base = pd.read_csv('iris.csv')
previsores = base.iloc[:, 0:4].values
classe = base.iloc[:, 4].values

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
classe = labelencoder.fit_transform(classe)

previsores = previsores.astype('float32')
classe = classe.astype('int64')

In [ ]:
clf_sklearn_tunning = NeuralNetClassifier(module = classificador_torch,
                                            criterion = torch.nn.CrossEntropyLoss,
                                            train_split = False)


In [ ]:
params = {'batch_size': [10, 30],
          'max_epochs': [2000, 3000],
          'optimizer': [torch.optim.Adam, torch.optim.SGD],
          'module__activation': [F.relu, F.tanh, F.sigmoid],
          'module__neurons': [4, 8, 16],
          'module__initializer': [torch.nn.init.uniform_, torch.nn.init.normal_],
          'module__dropout': [0.2, 0.3]
          }

In [ ]:
grid_search = GridSearchCV(estimator = clf_sklearn_tunning, 
                           param_grid = params, 
                           scoring = 'accuracy',
                           cv = 3)

In [ ]:
grid_search = grid_search.fit(previsores, classe)

A saída de streaming foi truncada nas últimas 5000 linhas.
    857        0.3642  0.0120
    858        0.3842  0.0118
    859        0.5559  0.0121
    860        0.3132  0.0122
    861        0.4438  0.0120
    862        0.4826  0.0123
    863        0.3522  0.0122
    864        0.3897  0.0121
    865        0.5592  0.0147
    866        0.4574  0.0122
    867        0.4054  0.0122
    868        0.3037  0.0123
    869        0.4130  0.0122
    870        0.3824  0.0120
    871        0.4794  0.0121
    872        0.4737  0.0120
    873        0.5082  0.0149
    874        0.5164  0.0123
    875        0.3919  0.0121
    876        0.4881  0.0121
    877        0.4939  0.0122
    878        0.5588  0.0123
    879        0.4371  0.0135
    880        0.4765  0.0124
    881        0.3806  0.0123
    882        0.4994  0.0122
    883        0.4942  0.0120
    884        0.4416  0.0122
    885        0.4400  0.0122
    886        0.4157  0.0119
    887        0.5134  0.0119
    888    

In [ ]:
melhores_parametros = grid_search.best_params_
melhor_precisao = grid_search.best_score_